In [1]:
import pandas as pd
from scipy import stats
import numpy as np

In [2]:
train = pd.read_csv('clean_train.csv')

In [4]:
x = list(train.columns[:7])
x.remove('Demand')
print(x)
for i in x:
    train[x] = train[x].astype('category')

['Stall_no', 'Market_Category', 'Loyalty_customer', 'Product_Category', 'Grade', 'Discount_avail']


In [5]:
numeric_ = list(train.columns[7:])
# numeric_.remove('Demand')
# numeric_.remove('charges_2 (%)')
numeric_.append('Demand')
print(numeric_)

['charges_1', 'charges_2 (%)', 'Minimum_price', 'Maximum_price', 'Selling_Price', 'Demand']


In [6]:
z = np.abs(stats.zscore(train[numeric_]))
txx = train[(z < 3).all(axis=1)]

In [7]:
train = txx

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6188 entries, 0 to 6367
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Stall_no          6188 non-null   category
 1   Market_Category   6188 non-null   category
 2   Loyalty_customer  6188 non-null   category
 3   Product_Category  6188 non-null   category
 4   Grade             6188 non-null   category
 5   Demand            6188 non-null   int64   
 6   Discount_avail    6188 non-null   category
 7   charges_1         6188 non-null   int64   
 8   charges_2 (%)     6188 non-null   int64   
 9   Minimum_price     6188 non-null   float64 
 10  Maximum_price     6188 non-null   float64 
 11  Selling_Price     6188 non-null   float64 
dtypes: category(6), float64(3), int64(3)
memory usage: 396.3 KB


In [12]:
x = train.iloc[:, :-1]
y = train.iloc[:, -1]

In [76]:
from sklearn.metrics import accuracy_score as asc 
from sklearn.model_selection import train_test_split as tts 
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.ensemble import VotingRegressor as vs

In [77]:
seed = 21

In [78]:
x_train, x_test, y_train, y_test = tts(x,y,test_size= 0.33, random_state= seed)
knn = KNeighborsRegressor()
dtr = DecisionTreeRegressor(random_state=seed)
rfg = RandomForestRegressor(random_state=seed)
las = Lasso(alpha=0.1, normalize = True)
classifier = [#('Logistic Regression', lr),
              ('KNeighborsRegressor', knn),
              ('DecisionTreeRegressor', dtr),
              ('RandomForestRegressor', rfg),
              ('Lasso', las)]

vreg = vs(estimators = classifier)

In [79]:
vreg.fit(x_train,y_train)

VotingRegressor(estimators=[('KNeighborsRegressor', KNeighborsRegressor()),
                            ('DecisionTreeRegressor',
                             DecisionTreeRegressor(random_state=21)),
                            ('RandomForestRegressor',
                             RandomForestRegressor(random_state=21)),
                            ('Lasso', Lasso(alpha=0.1, normalize=True))])

In [80]:
y_pred = vreg.predict(x_test)

In [81]:
print('VREG : ',vreg.score(x_test, y_test))

VREG :  0.9036947293734787
